In [ ]:
import cv2
import matplotlib.pyplot as plt
import pandas as pd
import os
import tensorflow as tf
import numpy as np
import time

In [ ]:
TRAIN_PATH = '/content/drive/MyDrive/rabbits/train'
TEST_PATH = '/content/drive/MyDrive/rabbits/test'
VAL_PATH = '/content/drive/MyDrive/rabbits/valid'

In [ ]:
cv2.setUseOptimized(True)
cv2.setNumThreads(4)

In [ ]:
def get_image(path):
    image = tf.io.read_file(path)
    image = tf.io.decode_jpeg(image, channels=3)
    return image.numpy()


def get_labels(path):
    df = pd.read_csv(os.path.join(path, '_annotations.csv'))
    labels = []
    images = []
    names = []
    
    for i in range(df.shape[0]):
        line = df.iloc[i].to_numpy()
        if line[0] in names:
            labels[-1].append(line[-4:])
            continue
        labels.append([line[-4:]])
        images.append(get_image(os.path.join(path, line[0])))
        names.append(line[0])


def unison_shuffled_copies(list_a, list_b):
    random_permutation = np.random.permutation(len(list_a))
    return list_a[random_permutation], list_b[random_permutation]


def iou(box1, box2):
    # box1 contains [min_x, min_y, max_x, max_y]
    # box2 contains [min_x, min_y, height, width]

    xmin = max(box1[1], box2[1])
    ymin = max(box1[0], box2[0])
    xmax = min(box1[3], box2[3] + box2[1])
    ymax = min(box1[2], box2[2] + box2[0])
    
    intersection = (xmax - xmin) * (ymax - ymin)
    union = (box1[3] - box1[1]) * (box1[2] - box1[0]) + box2[3] * box2[2] - intersection
    return max(0, intersection / union)


def get_proper_sets(x, y, target_shape=(224, 224)):
    negative_set = []
    positive_set = []
    max_height, max_width = x.shape[1:3]

    for counter, image in enumerate(x):
        boxes = y[counter]
        
        width = np.random.randint(50, target_shape[1])
        height = np.random.randint(50, target_shape[0])
        
        end_status = False
        for j in range(20):
            ix = np.random.randint(0, max_width - width)
            iy = np.random.randint(0, max_height - height)
            
            for num, box in enumerate(boxes):
                if iou(box, [iy, ix, height, width]) < 0.2:
                    if num == len(boxes) - 1:
                        end_status = True
                else:
                    break
            
            if end_status:
                break
        
        if not end_status:
            continue

        matr = image[iy:iy+height, ix:ix+width]
        negative_set.append(tf.image.resize(matr, target_shape).numpy())

        box = boxes[0]
        rect = image[box[1]:box[3], box[0]:box[2]]
        rect = tf.image.resize(rect, target_shape).numpy()
        positive_set.append(rect)
    
    positive_set, negative_set = np.array(positive_set), np.array(negative_set)

    set_x = np.vstack((positive_set, negative_set))
    set_y = np.concatenate([np.ones((positive_set.shape[0],)), np.zeros((negative_set.shape[0],))])
    set_x, set_y = unison_shuffled_copies(set_x, set_y)

    set_x = (set_x / 255).astype(np.float32)
    return set_x, set_y

In [ ]:
train_x, train_y = get_labels(TRAIN_PATH)
val_x, val_y = get_labels(VAL_PATH)

proper_train_x, proper_train_y = get_proper_sets(train_x, train_y)
proper_val_x, proper_val_y = get_proper_sets(val_x, val_y)

In [ ]:
vgg_layers = tf.keras.applications.VGG19(include_top=False, weights='imagenet', input_shape=(224, 224, 3))
vgg_layers.trainable = False

In [ ]:
flat = tf.keras.layers.Flatten()(vgg_layers.output)

dense1 = tf.keras.layers.Dense(128, activation='relu')(flat)
dense2 = tf.keras.layers.Dense(1, activation='sigmoid')(dense1)

model_vgg = tf.keras.Model(inputs=vgg_layers.input, outputs=dense2)

model_vgg.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=2e-3), loss='binary_crossentropy', metrics=['acc'])
model_vgg.summary()

In [ ]:
model_vgg.fit(proper_train_x, proper_train_y, epochs=2, validation_data=(proper_val_x, proper_val_y))

In [ ]:
model_vgg.save('/content/drive/MyDrive/model_vgg.h5')